In [163]:
# -*- coding: utf-8 -*-
import random
import numpy as np
import tensorflow as tf
import collections
import math
from six.moves import xrange
from konlpy.tag import Twitter
from six.moves import xrange

class Word2Vec:
    # 데이터 에서 대화만 가져오기 
    def load_file(self,filesrc):
        f = open(filesrc, 'rt', encoding='utf-16')
        name = ""
        talk = []
        speak = {}
        not_person = ['al', 'ot']
        talk_all = []
        while(True):
            data = f.readline() 
            if not data: break
    
            if("<u who=" in data or "<s n=" in data):
                # 발화자 구분
                if "who" in data:
                    name = data[8:10]
                    if name in not_person:
                        continue
                    if not (name in speak):
                       speak[name] = [] 
    
                # 대화 추출
                temp = data.split(">")
                s = ""
                for v in temp:
                    idx = v.find('<')
                    if idx < 2: continue
                    speak[name].append(v[:idx])
                    talk_all.append(v[:idx])
    
        return speak,talk_all

    
    def words_join(self,words,doc_index):
        word_idx = []
        doc_count = 0
        for index,lines in enumerate(words):
            if len(doc_index) > doc_count and index >= doc_index[doc_count]:
                doc_count += 1
            word_idx += [[v,index,doc_count] for v in lines.split(' ')]
            
        return word_idx
#         return " ".join(words).split(' ')
    
    def postaging(self,docs):
        twitter=Twitter()
        want = ['Exclamation','Adverb','Noun','Alpha','Verb','Adjective']
        stopword = ['어어','넷','만','것','타','최','태','개','홈','선','끼','각','번','하다','음','화', '이다',' ','다','더','포','제','저','여기','고','씬','첨','난','면','으루','네']
        pos = []
        
        for lines in docs:
            strs=""
            for t in twitter.pos(lines, norm=True, stem=True):
                if (t[1] in want) and (t[0] not in stopword):
                    strs = strs + " " + t[0]  
            pos.append(strs[1:])
        
        return pos
    
    # word_list를 불러오기전 함수 
    #def __init__(self, word_list,vector_size,load_list):
    
    def __init__(self,vector_size,load_list,skip_window=3,num_steps=50001):
        #데이터 로드
        self.talk_list = []
        speak_list = []
        doc_index = []
        
        #load는 리스트 형태  , 한번에 여러문서도가능하도록 --> 나중에는 db형식으로 교체 필요
        for path in load_list:
            _,talks = self.load_file(path)
            self.talk_list= self.talk_list + talks
            doc_index.append(len(self.talk_list)-1)
        
        
        # 포스테깅 및 stopword 처리 
        word_list = self.postaging(self.talk_list)
    
        
        #테스트 데이터로 확인식 보통 단어수 2000개를 넘지 못함 
        self.vocabulary_size = 4000# 원핫 인코딩 시킬 단어의 갯수   
        self.words = self.words_join(word_list,doc_index)
        print("len(words) : ",len(self.words))
        print("words : "," ".join(self.talk_list).split(' '))
        
        self.data, self.count, self.dictionary, self.reverse_dictionary = self.build_dataset(self.words[:,0])
        self.batch, self.labels = self.generate_batch(batch_size=8, num_skips=2, skip_window=1)
        self.batch_size = 32
        self.embedding_size = vector_size  # embedding vector의 크기. 아웃풋 크기 
        self.skip_window = skip_window     # 윈도우 크기 : 왼쪽과 오른쪽으로 얼마나 많은 단어를 고려할지를 결정.
        self.num_skips = 2         # 레이블(label)을 생성하기 위해 인풋을 얼마나 많이 재사용 할 것인지를 결정.
               
        self.num_sampled = 64    # sample에 대한 negative examples의 개수.
        self.num_steps = num_steps  # server 시 올리
        
        self.final_embeddings = self.train(self.reverse_dictionary,self.data)
        
        
    def output(self):
        return self.final_embeddings, self.data, self.count, self.dictionary, self.reverse_dictionary,self.words,self.talk_list
      
        
    def train(self,reverse_dictionary,data):
        graph = tf.Graph()
        with graph.as_default():
        
          # 트레이닝을 위한 인풋 데이터들
          train_inputs = tf.placeholder(tf.int32, shape=[self.batch_size])
          train_labels = tf.placeholder(tf.int32, shape=[self.batch_size, 1])
        
          # Ops and variables pinned to the CPU because of missing GPU implementation
          with tf.device('/cpu:0'):
            # embedding vectors 행렬을 랜덤값으로 초기화
              embeddings = tf.Variable(
                    tf.random_uniform([self.vocabulary_size, self.embedding_size], -1.0, 1.0))
            # 행렬에 트레이닝 데이터를 지정
              embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
              # NCE loss를 위한 변수들을 선언
              nce_weights = tf.Variable(tf.truncated_normal([self.vocabulary_size, self.embedding_size], stddev=1.0 / math.sqrt(self.embedding_size)))
              nce_biases = tf.Variable(tf.zeros([self.vocabulary_size]))
        
          # batch의 average NCE loss를 계산한다.
          # tf.nce_loss 함수는 loss를 평가(evaluate)할 때마다 negative labels을 가진 새로운 샘플을 자동적으로 생성한다.
          loss = tf.reduce_mean(
              tf.nn.nce_loss(weights=nce_weights,
                             biases=nce_biases,
                             labels=train_labels,
                             inputs=embed,
                             num_sampled=self.num_sampled,
                             num_classes=self.vocabulary_size))
        
          optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)
        
        
          # minibatch examples과 모든 embeddings에 대해 cosine similarity를 계산한다.
          norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
          normalized_embeddings = embeddings / norm

        with tf.Session(graph=graph) as session:
          # 트레이닝을 시작하기 전에 모든 변수들을 초기화한다.
            tf.initialize_all_variables().run()
            print("Initialized")
        
            average_loss = 0
            for step in xrange(self.num_steps):
                batch_inputs, batch_labels = self.generate_batch(self.batch_size, self.num_skips, self.skip_window)
                feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}
            
            # optimizer op을 평가(evaluating)하면서 한 스텝 업데이트를 진행한다.
                _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
                average_loss += loss_val
        
                if step % 2000 == 0:
                    if step > 0:
                        average_loss /= 2000
                    # 평균 손실(average loss)은 지난 2000 배치의 손실(loss)로부터 측정된다.
                    print("Average loss at step ", step, ": ", average_loss)
                    average_loss = 0
                
                final_embeddings = normalized_embeddings.eval(session=session)
                
        return final_embeddings
          
    
    def build_dataset(self,words):
      count = [['UNK', -1]]
      count.extend(collections.Counter(words).most_common(self.vocabulary_size - 1))
      dictionary = dict()
      for word, _ in count:
        dictionary[word] = len(dictionary)
      data = list()
      unk_count = 0
      for word in words:
        if word in dictionary:
          index = dictionary[word]
        else:
          index = 0  # dictionary['UNK']
          unk_count += 1
        data.append(index)
      count[0][1] = unk_count
      reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
      return data, count, dictionary, reverse_dictionary
    
    
    # Step 3: skip-gram model을 위한 트레이닝 데이터(batch)를 생성하기 위한 함수.
    def generate_batch(self,batch_size, num_skips, skip_window):
        data_index = 0
        #global data_index
        assert batch_size % num_skips == 0
        assert num_skips <= 2 * skip_window
        batch = np.ndarray(shape=(batch_size), dtype=np.int32)
        labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
        span = 2 * skip_window + 1 # [ skip_window target skip_window ]
        buffer = collections.deque(maxlen=span)
        
        for _ in range(span):
            buffer.append(self.data[data_index])
            data_index = (data_index + 1) % len(self.data)
        for i in range(batch_size // num_skips):
            target = skip_window  # target label at the center of the buffer
            targets_to_avoid = [ skip_window ]
            for j in range(num_skips):
                while target in targets_to_avoid:
                    target = random.randint(0, span - 1)
                targets_to_avoid.append(target)
                batch[i * num_skips + j] = buffer[skip_window]
                labels[i * num_skips + j, 0] = buffer[target]
            buffer.append(self.data[data_index])
            data_index = (data_index + 1) % len(self.data)
        return batch, labels


    def similarity(self,find_word,top_word):
        find_word_data = self.final_embeddings[self.dictionary[find_word]]
        sims = []
        for key,value in  self.dictionary.items():
            similarity = self.final_embeddings[value]
            cosine_similarity = np.dot(find_word_data, similarity)/(np.linalg.norm(find_word_data)* np.linalg.norm(similarity))
            sims.append([cosine_similarity,key])
        sims.sort(reverse=True)
        return sims[1:top_word+1]
    

In [164]:
# import Word2Vec


#load = ["6CM00079.txt","6CM00080.txt","6CM00082.txt","6CM00083.txt","6CM00088.txt","6CM00090.txt","6CM00092.txt","6CM00093.txt","6CM00094.txt","6CM00095.txt"]
# load = "sh_news3.txt"
load = ["6CM00079-학교.txt","6CM00080-군대.txt","6CM00082-정치.txt"]


# 호출 및 벡터 사이즈 설정 
vector_size = 10
skip_window = 5
num_steps = 50001
#word2vec = Word2Vec.Word2Vec(pos,vector_size)
word2vec = Word2Vec(vector_size,load,skip_window,num_steps)
final_embeddings, datas, count, dictionary, reverse_dictionary,words,talk_list = word2vec.output()
# 유사한 단어 불러오기 
#print(dictionary)
# result = word2vec.similarity("군대",100)
# print(result)


len(words) :  13904
words :  ['음~', '제가', '교생실습', '갔다', '온', '얘기', '해', '드릴게요,', '그래,', '되게', '궁금하다,', '아~', '그래', '갖구::,', '저희', '교생이', '되게', '많았었거든요::,', '그런데', '저는,', '인제,', '제가', '나이가', '제일', '어렸어요::,', '생각보다', '나이', '많은', '사람들이', '되게', '', '', '많았거든요?', '음', '음,', '언니는', '', '구오,', '구오학번,', '아유~,', '그런', '얘기를', '여기서', '하면,', '구오', '학번까지', '막~,', '음::,', '그래', '갖구,', '근데,', '저는', '졸업한지', '사', '년밖에', '안', '돼서::,', '나도', '되게', '어리다고', '생각을', '했었는데,', '애들을', '보니까', '세대', '차이가', '엄청', '나는', '거예요,', '음::,', '걔네들은', '정말,', '걔들이', '메일을', '보내면', '진짜', '못', '읽겠는', '거예요', '막,', '난', '읽을', '줄', '아는데,', '머~', '아리용,', '아리용이', '무슨', '뜻이에요?', '아리용?', '음', '음,', '선생님', '아리용', '', '맨날', '그러는데', '무슨', '말인지', '몰라', '가지고,', '모르겠다,', '사투리', '아니야?', '어,', '아니에요,', '아니거든요,', '선생님', '아룡?', '어,', '아룡?', '어,', '아리용::', '맨날,', '아리영', '아리영', '그거', '아니야?', '인어', '아가씨에', '나오는', '거?', '그래', '가지고', '맨날', '그러고,', '어,', '특수문자도', '엄청', '많이', '쓰고,', '문자를', '보내도', '알아볼', '수가', '없는,', '음::,', '막', '그런', '문자를', '쓰고,', '

TypeError: list indices must be integers or slices, not tuple

In [171]:
test = [['교생실습', 0, 0], ['가다', 0, 0], ['온', 0, 0]]
test = np.array(test)
test = test[:,[0]]
print(test)
print(" ".join(test))
# print(" ".join(test[:,[0]]).split(' '))


[['교생실습']
 ['가다']
 ['온']]


TypeError: sequence item 0: expected str instance, numpy.ndarray found